## ¿Qué hace este script?

#### Para cada AP:
* usuarios.usuariosConectados(corresponde al total de conexiones), usuarios.sesiones_Usuarios (Corresponde al total de usuarios)
* Información detallada de los dispositivos conectados en el rango de fecha procesado: usuarios.dispositivo.mac, usuarios.dispositivo.tipo, usuarios.dispositivo.marca, usuarios.dispositivo.tecnologia, usuarios.dispositivo.sisOperativo.

#### Para cada sitio: 
* Totales por características de dispositivos: usuarios.sistemaOperativoUsuarios, usuarios.tipoDispositivoUsuarios, usuarios.marcaTerminal, usuarios.tecnologiaTerminal, usuarios.totales.dispositivos (Este ultimo guarda el total)
* usuarios.usoServicioInternetSitio es la suma en Gb consumido discriminado por usuarios.detallesTecnologiasTerminales (Bandas 2.4 o 5 GHz)
* usuarios.conteoDispositivos el cual indica el total de dispositivos conectados (Es lo mismo que total de conexiones)
* usuarios.usuariosNuevos (Esta tenía antes Nuevos y Recurrentes, pero ahora solo tiene Nuevos por solicitud de BI)

In [1]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import random
import re
import time

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=133, max_retries=3, retry_on_timeout=True
)

Función para realizar consultas cuando la cantidad de registros es mayor a 10.000

In [3]:
def custom_scan(query, index, total_docs, client):
    
    results = helpers.scan(client, index=index, query=query)
    
    data = []
    for item in results:
        data.append(item['_source'])
        if len(data) >= total_docs:
            break
            
    data = pd.DataFrame(data)
    print(data.shape)
    print(data)
    return data
    # return pd.DataFrame(data)

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [4]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### nombre de indice donde se insertará

In [5]:
indice = parametros.usuarios_tablero11_index 
indice_control = parametros.tableros_mintic_control

### Funcion para construir JSON compatible con ElasticSearch

In [6]:
def filterKeys(document, use_these_keys):
    return {key: document.get(key) for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [7]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["tablero11.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero11"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero11.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-06 00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:
* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud,COD_ISO, id_Beneficiario).

In [8]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud', 'COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    datos_semilla['site_id'] = datos_semilla['site_id'].str.strip()
except:
    exit()

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location<br>
* Se renombran los campos de semilla

In [9]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrÃ³n que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [10]:
datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})

Se descartan los registros que tengan la latitud y longitud vacía o no valida

In [11]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["usuarios.location"]=='')].index)

### leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR
* site_id para cruzar con las misma llave de semilla.
* datos del dispositivo: mac, status, ip.
* ap_group para identificar si la conexión es indoor/outdoor

In [12]:
try:
    query = {
        "_source": ["site_id","mac","status","ip","ap_group"], 
        "query": {
            "match_all": {}
        }
    }
    
    datos_dev = custom_scan(
        query, 
        parametros.cambium_d_d_index,
        total_docs=30000, 
        client=es
    )
    datos_dev['site_id'] = datos_dev['site_id'].str.strip()
    
except:
    exit()

(21076, 5)
                   ap_group              ip        site_id                mac  \
0      INDOOR-44745-ZGYO227  172.28.118.179  44745-ZGYO227  BC:E6:7C:5E:66:88   
1                   OUTDOOR  172.28.118.180  44745-ZGYO227  58:C1:7A:E9:C9:6F   
2                   OUTDOOR  172.28.118.181  44745-ZGYO227  58:C1:7A:E9:C8:5C   
3      INDOOR_44747-ZGYO413  172.28.118.171  44747-ZGYO413  BC:E6:7C:ED:F2:4F   
4                   OUTDOOR  172.28.118.172  44747-ZGYO413  BC:E6:7C:4F:07:88   
...                     ...             ...            ...                ...   
21071               OUTDOOR  172.25.227.228  52005-ZCKE815  BC:A9:93:69:B0:A8   
21072               OUTDOOR  172.25.227.229  52005-ZCKE815  BC:A9:93:6A:2C:B8   
21073  INDOOR_52005-ZCKE815  172.25.227.227  52005-ZCKE815  BC:E6:7C:5D:88:57   
21074               OUTDOOR  172.25.229.197  19327-ZCKE884  BC:A9:93:67:75:B0   
21075                                        21836-ZGYO922  58:C1:7A:18:B3:20   

           statu

Se descartan registros con site_id vacios ya que no cruzarán en el merge y se limpian los NaN del dataframe.

In [13]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

In [14]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split(".", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos unicos con mac.

In [15]:
datos_dev = datos_dev.drop_duplicates('mac')

Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario 

In [16]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

## Lectura de datos ohmyfi-detalleconexiones

Los datos que se toman son:
* fechahora (de cada conexión). fecha_control es lo mismo pero con el intervalo de 10 minutos
* Información del centro: lugar, lugar_cod.
* Información del dispositivo de usuario: mac_usuario, dispositivo, sistema_operativo.
* Información de usuario: tipodoc y documento

In [17]:
def trae_conexiones(fecha_ini, fecha_fin, client):
    query = {
        "_source": [
            "fechahora", "fecha_control", "lugar", "lugar_cod", 
            "mac_usuario", "dispositivo", "sistema_operativo", 
            'tipodoc', 'documento'
        ],
        "query": {
            "range": {
                "fechahora": {
                    "gte": fecha_ini,
                    "lt": fecha_fin
                }
            }
        }
    }
    return custom_scan(
        query, 
        parametros.ohmyfi_d_c_index,
        total_docs=5000000, 
        client=client
    )

### Lanzando ejecución de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 1 dia
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.


In [18]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_det_conex = trae_conexiones(fecha_max_mintic, fecha_tope_mintic, es)

if datos_det_conex is None or datos_det_conex.empty:
    while (datos_det_conex is None or datos_det_conex.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_det_conex = trae_conexiones(fecha_max_mintic, fecha_tope_mintic, es)
else:
    pass

(82165, 9)
                 fechahora        mac_usuario dispositivo  \
0      2021-06-06 10:30:00  10:89:FB:B6:9E:66  Smartphone   
1      2021-06-06 10:30:01  14:5F:94:FF:12:34  Smartphone   
2      2021-06-06 10:30:01  10:89:FB:B6:9E:66  Smartphone   
3      2021-06-06 10:30:02  C8:14:51:96:33:05  Smartphone   
4      2021-06-06 10:30:00  10:89:FB:B6:9E:66  Smartphone   
...                    ...                ...         ...   
82160  2021-06-06 23:07:24  5C:77:76:5B:CB:4D  Smartphone   
82161  2021-06-06 23:07:25  30:4B:07:FB:8D:35  Smartphone   
82162  2021-06-06 23:07:25  68:A0:3E:90:79:E9  Smartphone   
82163  2021-06-06 23:07:27  26:DF:87:33:E2:99  Smartphone   
82164  2021-06-06 23:07:34  26:DF:87:33:E2:99  Smartphone   

                                          lugar   documento tipodoc  \
0                           EL JABO, VALLEDUPAR  1065602326      TI   
1                                  VDA DELICIAS    51972989      CC   
2                           EL JABO, VALLED

In [19]:
datos_det_conex['lugar_cod'] = datos_det_conex['lugar_cod'].str.strip()

In [20]:
datos_det_conex['fecha'] = datos_det_conex['fecha_control'].str.split(" ", n = 1, expand = True)[0]
datos_det_conex.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

In [21]:
datos_det_conex = datos_det_conex.rename(columns={'lugar_cod' : 'site_id'
                                                   ,'fechahora':'usuarios.fechaConexionUsuarios'
                                                   ,'dispositivo': 'usuarios.tipoDispositivoUsuarios'
                                                   , 'sistema_operativo': 'usuarios.sistemaOperativoUsuarios'})

Se corrigen valores errados de site_id en detalle conexiones

## Lectura de datos ohmyfi-consumo

In [22]:
def traeSesiones(fecha_max, fecha_tope, client):
    
    query = {
        "_source": [
            "lugar_cod", "tiempo_sesion_minutos", 
            "mac_ap", "fecha_inicio"
        ], 
        "query": {
            "range": {
                "fecha_inicio": {
                    "gte": fecha_max,
                    "lt": fecha_tope
                }
            }
        }
    }

    return custom_scan(
        query, 
        parametros.ohmyfi_consumos_index,
        total_docs=500000, 
        client=client
    )

## Lectura de datos all cambium device client

### Se lee la información de cambium device client

In [23]:
def traeRadio(fecha_max, fecha_tope, client):
        
    query = {
        "_source": [
            'mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 
            'radio.tx_bytes', 'fecha_control'
        ], 
        "query": {
            "range": {
                "fecha_control": {
                    "gte": fecha_max,
                    "lt": fecha_tope 
                }
            }
        }
    }
        
    return custom_scan(
        query, 
        'all-' + parametros.cambium_d_c_index,
        total_docs=100000, 
        client=client
    )

Esta lectura se usa para identificar: <br>
* Lecturas de consumo por dispositivo (radios rx y tx)<br>
* Fabricante del dispositivo<br>
* la mac del ap que luego se usa para identificar el ap group(Indoor/Outdoor)

In [24]:
query = {
    "_source": [
        "mac", "ap_mac", 'manufacturer', 'name', 
        "radio.band", 'radio.rx_bytes', 'radio.tx_bytes'
    ],
    "query": {
        "bool": {
            "filter": [{
                "exists": {
                    "field":"mac"
                }
            }]
        }
    }
}

datos_dev_clients = custom_scan(
    query, 
    parametros.cambium_d_c_index,
    total_docs=30000000, 
    client=es
)

(932042, 7)
                   ap_mac                      name                mac  \
0       BC:E6:7C:EC:DD:FC           HUAWEI_Y7a-3b52  0E:9C:F2:E0:B3:1F   
1       BC:E6:7C:5D:20:AE                 SM-J260MU  F8:F1:E6:A3:0D:45   
2       BC:A9:93:00:98:33           android-e7938e7  00:08:22:46:78:82   
3       BC:E6:7C:EC:AA:DC                Galaxy-A01  A0:27:B6:62:78:47   
4       BC:E6:7C:ED:8B:98            carlosl74-miPC  4C:ED:DE:87:23:33   
...                   ...                       ...                ...   
932037  BC:E6:7C:E7:FC:D3  android-d182e728db4c75df  18:D6:1C:A4:86:02   
932038  BC:A9:93:10:86:55  android-1ad84b38ff5ac6cc  DC:CF:96:1D:89:19   
932039  BC:E6:7C:E7:FC:6D  android-1b0f3c487aae1436  54:F2:01:8E:BF:65   
932040  BC:E6:7C:E7:FC:D3                        NA  60:1D:91:66:52:19   
932041  BC:E6:7C:4E:3D:C6                        NA  72:4F:2D:61:F3:6A   

       radio.band  radio.rx_bytes  radio.tx_bytes  \
0          2.4GHz            1140             

* se cruza por mac_usuario<br>
* Los merge usan left para evitar perdida de datos en cruce con cambium-devicedevices. Aquellos datos que no cruzan se les marca como no identificados. En condiciones ideales, no deberÃ­a presentarse ausencia de informaciÃ³n<br>
* Solo se toma lo que cruza con INNER<br>
* se cambia el fabricante [Local MAC]<br>
* se cambian lo nulos

In [25]:
datos_dev_clients.drop_duplicates(inplace=True)

In [26]:
try:
    datos_dev_clients.drop_duplicates(inplace=True)
    datos_dev_clients = datos_dev_clients.rename(columns={'ap_mac' : 'usuarios.macRed','mac' : 'mac_usuario'})
    datos_dev_clients['manufacturer'] = datos_dev_clients['manufacturer'].replace("[Local MAC]", "No identificado")
    datos_dev_clients.fillna({'manufacturer':'No identificado'},inplace=True)
    datos_det_conex = pd.merge(datos_det_conex,  datos_dev_clients, on='mac_usuario',how='left')
    datos_det_conex = pd.merge(datos_det_conex,datos_dev[['usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    datos_det_conex.fillna({'usuarios.tipoDispositivoUsuarios':'No identificado'
                       ,'usuarios.sistemaOperativoUsuarios':'No identificado'
                       ,'manufacturer':'No identificado'
                       ,'radio.band':'No identificado'
                       },inplace=True)
except:
    pass

## Calculando totales por dispositivos

La lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [27]:
use_these_keys = [ 'usuarios.tablero11.siteID'
                  , 'usuarios.tablero11.nomCentroDigital'
                  , 'usuarios.tablero11.nombreDepartamento'
                  , 'usuarios.tablero11.nombreMunicipio'
                  , 'usuarios.tablero11.idBeneficiario'
                  , 'usuarios.tablero11.location'
                  , 'usuarios.tablero11.fechaControl'
                  , 'usuarios.tablero11.fecha'
                  , 'usuarios.tablero11.anyo'
                  , 'usuarios.tablero11.mes'
                  , '@timestamp']

def doc_generator(df,use_these_keys):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{ 'totalesDispositivos-' + str(document['usuarios.tablero11siteID']) + '-' + str(document['usuarios.fechaControl'])+ '-' + str(document['usuarios.tablero11.fechaControl'])+'-'+str(random.randrange(10000))}",
                "_source": filterKeys(document,use_these_keys),
            }

Se agrupa por:
* Sistema operativo
* Tipo dispositivo
* Marca del terminal
* Tecnología terminal

Se cuentan las mac de usuario para cada site id y fecha control

In [28]:
def cambioVariable(df):
    df = df.rename(columns={'siteID':'usuarios.tablero11.siteID'
                        ,'usuarios.siteID': 'usuarios.tablero11.siteID'
                        ,'usuarios.tablero11siteID':'usuarios.tablero11.siteID'
                        ,'fechaControl':'usuarios.tablero11.fechaControl'
                        ,'usuarios.fechaControl':'usuarios.tablero11.fechaControl'
                        ,'usuarios.tablero11fechaControl': 'usuarios.tablero11.fechaControl'
                        ,'nomCentroDigital':'usuarios.tablero11.nomCentroDigital'
                        ,'usuarios.nomCentroDigital':'usuarios.tablero11.nomCentroDigital'
                        ,'usuarios.tablero11nomCentroDigital':'usuarios.tablero11.nomCentroDigital'
                        ,'localidad':'usuarios.tablero11.localidad'
                        ,'usuarios.localidad':'usuarios.tablero11.localidad'
                        ,'usuarios.tablero11localidad':'usuarios.tablero11.localidad'
                        ,'nombreDepartamento':'usuarios.tablero11.nombreDepartamento'
                        ,'usuarios.nombreDepartamento':'usuarios.tablero11.nombreDepartamento'
                        ,'usuarios.tablero11nombreDepartamento':'usuarios.tablero11.nombreDepartamento'
                        ,'nombreMunicipio':'usuarios.tablero11.nombreMunicipio'
                        ,'usuarios.nombreMunicipio':'usuarios.tablero11.nombreMunicipio'
                        ,'usuarios.tablero11nombreMunicipio':'usuarios.tablero11.nombreMunicipio'
                        ,'idBeneficiario':'usuarios.tablero11.idBeneficiario'
                        ,'usuarios.idBeneficiario':'usuarios.tablero11.idBeneficiario'
                        ,'usuarios.tablero11idBeneficiario':'usuarios.tablero11.idBeneficiario'
                        ,'apGroup':'usuarios.tablero11.apGroup'
                        ,'usuarios.apGroup':'usuarios.tablero11.apGroup'
                        ,'usuarios.tablero11apGroup':'usuarios.tablero11.apGroup'
                        ,'location':'usuarios.tablero11.location'
                        ,'usuarios.location':'usuarios.tablero11.location'
                        ,'usuarios.tablero11location':'usuarios.tablero11.location'
                        ,'fecha':'usuarios.tablero11.fecha'
                        ,'usuarios.fecha':'usuarios.tablero11.fecha'
                        ,'usuarios.tablero11fecha':'usuarios.tablero11.fecha'
                        ,'anyo':'usuarios.tablero11.anyo'
                        ,'usuarios.anyo':'usuarios.tablero11.anyo'
                        ,'usuarios.tablero11anyo':'usuarios.tablero11.anyo'
                        ,'mes':'usuarios.tablero11.mes'
                        ,'usuarios.mes':'usuarios.tablero11.mes'
                        ,'usuarios.tablero11mes':'usuarios.tablero11.mes'
                        ,'macRed':'usuarios.tablero11.macRed'
                        ,'usuarios.macRed':'usuarios.tablero11.macRed'
                        ,'usuarios.tablero11macRed':'usuarios.tablero11.macRed'
                        ,'usuariosConectados':'usuarios.tablero11.usuariosConectados'
                        ,'usuarios.usuariosConectados':'usuarios.tablero11.usuariosConectados'
                        ,'usuarios.tablero11usuariosConectados':'usuarios.tablero11.usuariosConectados'
                        ,'sesiones_Usuarios':'usuarios.tablero11.sesiones_Usuarios'
                        ,'usuarios.sesiones_Usuarios':'usuarios.tablero11.sesiones_Usuarios'
                        ,'usuarios.tablero11sesiones_Usuarios':'usuarios.tablero11.sesiones_Usuarios'
                        ,'consumoUsuarios':'usuarios.tablero11.consumoUsuarios'
                        ,'usuarios.consumoUsuarios':'usuarios.tablero11.consumoUsuarios'
                        ,'usuarios.tablero11consumoUsuarios':'usuarios.tablero11.consumoUsuarios'
                        ,'consumoUsuariosDescarga_aux':'usuarios.tablero11.consumoUsuariosDescarga_aux'
                        ,'usuarios.consumoUsuariosDescarga_aux':'usuarios.tablero11.consumoUsuariosDescarga_aux'
                        ,'usuarios.tablero11consumoUsuariosDescarga_aux':'usuarios.tablero11.consumoUsuariosDescarga_aux'
                        ,'consumoUsuariosCarga_aux':'usuarios.tablero11.consumoUsuariosCarga_aux'
                        ,'usuarios.consumoUsuariosCarga_aux':'usuarios.tablero11.consumoUsuariosCarga_aux'
                        ,'usuarios.tablero11consumoUsuariosCarga_aux':'usuarios.tablero11.consumoUsuariosCarga_aux'
                        ,'consumoUsuariosDescarga':'usuarios.tablero11.consumoUsuariosDescarga'
                        ,'usuarios.consumoUsuariosDescarga':'usuarios.tablero11.consumoUsuariosDescarga'
                        ,'usuarios.tablero11consumoUsuariosDescarga':'usuarios.tablero11.consumoUsuariosDescarga'
                        ,'consumoUsuariosCarga': 'usuarios.tablero11.consumoUsuariosCarga'
                        ,'usuarios.tablero11consumoUsuariosCarga': 'usuarios.tablero11.consumoUsuariosCarga'
                       }
              )
    
    df_vacio = pd.DataFrame(index=df.index)
    
    col_select= ['usuarios.tablero11.siteID'
                 ,'usuarios.tablero11.nomCentroDigital'
                 , 'usuarios.tablero11.localidad'
                 , 'usuarios.tablero11.nombreDepartamento'
                 , 'usuarios.tablero11.nombreMunicipio'
                 , 'usuarios.tablero11.idBeneficiario'
                 , 'usuarios.tablero11.location'
                 , 'usuarios.tablero11.sesiones_Usuarios'
                 , 'usuarios.tablero11.usuariosConectados'
                 , 'usuarios.tablero11.consumoUsuarios'
                 , 'usuarios.tablero11.consumoUsuariosDescarga_aux'
                 , 'usuarios.tablero11.consumoUsuariosCarga_aux'
                 , 'usuarios.tablero11.consumoUsuariosDescarga'
                 , 'usuarios.tablero11.consumoUsuariosCarga'
                 , 'usuarios.tablero11.apGroup'
                 , 'usuarios.tablero11.fechaControl'
                 , 'usuarios.tablero11.fecha'
                 , 'usuarios.tablero11.anyo'
                 , 'usuarios.tablero11.mes'
                 , 'usuarios.tablero11.macRed'
                 , '@timestamp']
    
    for c in col_select:
        try:
            df_vacio=pd.concat([df_vacio,df[c]],axis=1)
        except:
            pass
    
    df_vacio=df_vacio.iloc[:,~df_vacio.columns.duplicated()]
    return df_vacio


In [29]:
try:
    totalesDispositivos = datos_det_conex[["fecha_control","site_id"
                                 ,"usuarios.sistemaOperativoUsuarios"
                                 ,'usuarios.tipoDispositivoUsuarios'
                                 ,'manufacturer'
                                 ,'radio.band'
                                 ,"mac_usuario"]].groupby(["fecha_control","site_id"
                                                     ,"usuarios.sistemaOperativoUsuarios"
                                                     ,'usuarios.tipoDispositivoUsuarios'
                                                     ,'manufacturer'
                                                     ,'radio.band']).agg(['count']).reset_index()
    totalesDispositivos.columns = totalesDispositivos.columns.droplevel(1)
    totalesDispositivos= totalesDispositivos.rename(columns={'mac_usuario' : 'usuarios.totales.dispositivos'
                                                            ,'fecha_control' : 'usuarios.fechaControl'
                                                            ,'manufacturer' : 'usuarios.marcaTerminal'
                                                            ,'radio.band' : 'usuarios.tecnologiaTerminal'
                                                            })
    totalesDispositivos = pd.merge(totalesDispositivos, datos_semilla, on='site_id',how='inner')
    totalesDispositivos["usuarios.fecha"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    totalesDispositivos["usuarios.anyo"] = totalesDispositivos["usuarios.fecha"].str[0:4]
    totalesDispositivos["usuarios.mes"] = totalesDispositivos["usuarios.fecha"].str[5:7]
    totalesDispositivos["usuarios.dia"] = totalesDispositivos["usuarios.fecha"].str[8:10]
    totalesDispositivos["usuarios.hora"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    totalesDispositivos["usuarios.minuto"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    totalesDispositivos= totalesDispositivos.rename(columns={'site_id' : 'usuarios.siteID'})
    totalesDispositivos['nombreDepartamento'] = totalesDispositivos['usuarios.nombreDepartamento']
    totalesDispositivos['nombreMunicipio'] = totalesDispositivos['usuarios.nombreMunicipio']
    totalesDispositivos['idBeneficiario'] = totalesDispositivos['usuarios.idBeneficiario']
    totalesDispositivos['fecha'] = totalesDispositivos['usuarios.fecha']
    totalesDispositivos['anyo'] = totalesDispositivos['usuarios.anyo']
    totalesDispositivos['mes'] = totalesDispositivos['usuarios.mes']
    totalesDispositivos['dia'] = totalesDispositivos['usuarios.dia']
    totalesDispositivos['@timestamp'] = now.isoformat()
    
    totalesDispositivos = cambioVariable(totalesDispositivos)

except:
    pass

# Insertando usuarios conectados al indice principal

* Se calcula cantidad de sesiones por sitio con detalle conexiones, pero contando la ocurrencia unica de documento<br>
* Se calcula la cantidad de conexiones. Se agrupa por el campo usuarios.macRed(el cual corresponde al AP) y fecha_control. Se cuenta las ocurrencias de mac_usuario. Luego al cruzar con flujo principal, si el dato es nulo para ese momento, se debe colocar en 0.

In [30]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic, fecha_tope_mintic, es)

try:

    if datos_consumos is None or datos_consumos.empty:
        while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S"').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
            fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
            datos_consumos = traeSesiones(fecha_max_mintic, fecha_tope_mintic, es)
    else:
        pass
except:
    pass

(154651, 4)
               fecha_inicio  tiempo_sesion_minutos             mac_ap  \
0       2021-06-06 00:01:37                 120.00  58:C1:7A:E8:77:EA   
1       2021-06-06 00:30:27                   3.18  58:C1:7A:E8:77:EA   
2       2021-06-06 00:33:39                   2.25  58:C1:7A:E8:76:8B   
3       2021-06-06 00:35:53                   2.43  58:C1:7A:E8:77:EA   
4       2021-06-06 00:38:20                  14.15  58:C1:7A:E8:76:8B   
...                     ...                    ...                ...   
154646  2021-06-06 23:30:11                   1.83  58:C1:7A:E8:C4:44   
154647  2021-06-06 23:32:02                   0.15  58:C1:7A:E8:C4:6F   
154648  2021-06-06 23:32:11                   0.17  58:C1:7A:E8:C4:44   
154649  2021-06-06 23:32:21                  24.85  58:C1:7A:E8:C4:6F   
154650  2021-06-06 23:00:10                   0.05  BC:E6:7C:4E:3E:32   

            lugar_cod  
0       22483-ZGYO057  
1       22483-ZGYO057  
2       22483-ZGYO057  
3       22483-Z

### Se lee el indice all-cambium-device-client

* En este indice se guarda el detalle de los radio por fecha<br>
* Detalle conexiones cruza con device clients. Con estos se calculan los totales por marca

In [31]:
query = {
    "_source": [
        'mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 
        'radio.tx_bytes','fecha_control'
    ], 
    "query": {
        "range": {
            "fecha_control": {
                "gte": fecha_max_mintic,
                "lt": fecha_tope_mintic 
            }
        }
    }
}

datos_radio = custom_scan(
    query, 
    'all-' + parametros.cambium_d_c_index,
    total_docs=100000, 
    client=es
)

datos_performance = datos_radio

(100000, 6)
                  ap_mac        fecha_control                mac radio.band  \
0      BC:E6:7C:E7:FD:12  2021-06-06 00:02:00  24:46:C8:85:38:BA     2.4GHz   
1      BC:E6:7C:E8:CE:D9  2021-06-06 00:02:00  C0:D3:C0:B0:69:8B     2.4GHz   
2      BC:E6:7C:EF:FD:F1  2021-06-06 00:02:00  A0:4C:5B:59:FF:AD     2.4GHz   
3      BC:E6:7C:4E:3D:5D  2021-06-06 00:02:00  F8:F1:E6:F4:CA:01     2.4GHz   
4      BC:E6:7C:58:E6:CA  2021-06-06 00:02:00  90:97:F3:5E:BD:CC     2.4GHz   
...                  ...                  ...                ...        ...   
99995  BC:E6:7C:5B:7B:D7  2021-06-06 15:38:00  FC:DE:90:9C:1E:63     2.4GHz   
99996  BC:E6:7C:EE:2D:6F  2021-06-06 15:38:00  40:78:6A:C9:B3:6C     2.4GHz   
99997  BC:E6:7C:5B:7D:04  2021-06-06 15:38:00  88:46:04:DF:8B:38     2.4GHz   
99998  BC:E6:7C:5B:AF:F5  2021-06-06 15:38:00  3C:05:18:3C:26:19     2.4GHz   
99999  BC:E6:7C:5E:33:3E  2021-06-06 15:38:00  82:3B:21:F3:67:4D     2.4GHz   

       radio.rx_bytes  radio.tx_bytes  

In [32]:
if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traeRadio(fecha_max_mintic, fecha_tope_mintic, es)
else:
    pass

In [33]:
try:
    datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento','usuarios.macRed']].groupby(["fecha_control","site_id","usuarios.macRed"])['documento'].nunique().reset_index()
    datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})
    
    ################# datos_performance #####################
    datos_performance['fecha_control'] = datos_performance["fecha_control"].str[0:-4] + '0:00'
    datos_performance.replace('','0',inplace=True)
    datos_performance.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
    datos_performance['radio.rx_bytes'] = datos_performance['radio.rx_bytes'].astype(float)
    datos_performance['radio.tx_bytes'] = datos_performance['radio.tx_bytes'].astype(float)
    
    datos_performance = datos_performance.groupby(['ap_mac', 'fecha_control', 'mac', 'radio.band']).agg(['max']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance[['ap_mac'
                                         , 'fecha_control'
                                         , 'radio.rx_bytes'
                                         , 'radio.tx_bytes']].groupby(['ap_mac'
                                                                     , 'fecha_control']).agg(['sum']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance.rename(columns={'radio.rx_bytes': 'usuarios.consumoUsuariosDescarga_aux'
                                                     , 'radio.tx_bytes':'usuarios.consumoUsuariosCarga_aux'})
    
    datos_performance['usuarios.consumoUsuariosDescarga'] = round((datos_performance['usuarios.consumoUsuariosDescarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuariosCarga'] = round((datos_performance['usuarios.consumoUsuariosCarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuarios'] = datos_performance['usuarios.consumoUsuariosDescarga'] + datos_performance['usuarios.consumoUsuariosCarga']
    datos_performance['usuarios.consumoUsuarios'] = round(datos_performance['usuarios.consumoUsuarios'],6)
    datos_performance = datos_performance.rename(columns={'ap_mac':'usuarios.macRed'})
    
    
    datos_performance = pd.merge(datos_performance,datos_dev[['site_id','usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    
    aux_performance = pd.merge(datos_performance,  datos_semilla, on='site_id',how='inner')
    aux_performance.fillna({'usuarios.consumoUsuariosDescarga':0,
                              'usuarios.consumoUsuariosCarga':0,
                              'usuarios.consumoUsuarios':0,
                              'usuarios.apGroup':'No identificado'},inplace=True)
    aux_performance = aux_performance.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    aux_performance["usuarios.fecha"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    aux_performance["usuarios.anyo"] = aux_performance["usuarios.fecha"].str[0:4]
    aux_performance["usuarios.mes"] = aux_performance["usuarios.fecha"].str[5:7]
    aux_performance["usuarios.dia"] = aux_performance["usuarios.fecha"].str[8:10]
    aux_performance["usuarios.hora"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    aux_performance["usuarios.minuto"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    aux_performance['nombreDepartamento'] = aux_performance['usuarios.nombreDepartamento']
    aux_performance['nombreMunicipio'] = aux_performance['usuarios.nombreMunicipio']
    aux_performance['idBeneficiario'] = aux_performance['usuarios.idBeneficiario']
    aux_performance['fecha'] = aux_performance['usuarios.fecha']
    aux_performance['anyo'] = aux_performance['usuarios.anyo']
    aux_performance['mes'] = aux_performance['usuarios.mes']
    aux_performance['dia'] = aux_performance['usuarios.dia']
    
    
    ##################
    usuariosConectados = datos_det_conex[["fecha_control","site_id","usuarios.macRed","mac_usuario"]].groupby(["fecha_control","site_id","usuarios.macRed"]).agg(['count']).reset_index()
    usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
    usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.tablero11usuariosConectados'})
    
    ###################
    usuariosConectados = pd.merge(usuariosConectados,datos_logins, on=["fecha_control","site_id","usuarios.macRed"], how='inner')
    
    usuariosConectados= usuariosConectados.rename(columns={'usuarios.sesiones_Usuarios' : 'usuarios.tablero11sesiones_Usuarios'})
    
    ###################
    usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
    
   
    #Al parecer no esta viniendo consumoUsuario....
    
    usuariosConectados.fillna({'usuarios.consumoUsuarios' : 0
                              ,'usuarios.consumoUsuariosDescarga':0
                              ,'usuarios.consumoUsuariosCarga':0},inplace=True)
    
    ####################
    usuariosConectados = pd.merge(usuariosConectados,datos_dev[['usuarios.apGroup','usuarios.macRed']], on ='usuarios.macRed', how='left')
    
    usuariosConectados.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    usuariosConectados['usuarios.tablero11usuariosConectados'] = usuariosConectados['usuarios.tablero11usuariosConectados'].astype(int)
    usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    usuariosConectados["usuarios.fecha"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    usuariosConectados["usuarios.anyo"] = usuariosConectados["usuarios.fecha"].str[0:4]
    usuariosConectados["usuarios.mes"] = usuariosConectados["usuarios.fecha"].str[5:7]
    usuariosConectados["usuarios.dia"] = usuariosConectados["usuarios.fecha"].str[8:10]
    usuariosConectados["usuarios.hora"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    usuariosConectados["usuarios.minuto"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    usuariosConectados['nombreDepartamento'] = usuariosConectados['usuarios.nombreDepartamento']
    usuariosConectados['nombreMunicipio'] = usuariosConectados['usuarios.nombreMunicipio']
    usuariosConectados['idBeneficiario'] = usuariosConectados['usuarios.idBeneficiario']
    usuariosConectados['fecha'] = usuariosConectados['usuarios.fecha']
    usuariosConectados['anyo'] = usuariosConectados['usuarios.anyo']
    usuariosConectados['mes'] = usuariosConectados['usuarios.mes']
    usuariosConectados['dia'] = usuariosConectados['usuarios.dia']
    
    #####################
    columnas_aux_performance=[ 'usuarios.fechaControl', 'usuarios.siteID',                               'usuarios.apGroup', 'usuarios.macRed',                               'usuarios.consumoUsuariosCarga_aux', 'usuarios.consumoUsuariosDescarga',                               'usuarios.consumoUsuariosCarga', 'usuarios.consumoUsuarios',                                 'usuarios.consumoUsuariosDescarga_aux']
    columnas_merge = ['usuarios.fechaControl','usuarios.siteID','usuarios.apGroup','usuarios.macRed']
    
    usuariosConectados=pd.merge(usuariosConectados,aux_performance[columnas_aux_performance],on=columnas_merge, how='left')
    
    usuariosConectados = usuariosConectados.rename(columns=
                                                   {'usuarios.consumoUsuariosCarga_aux':'usuarios.tablero11consumoUsuariosCarga_aux'
                                                    ,'usuarios.consumoUsuariosDescarga_aux':'usuarios.tablero11consumoUsuariosDescarga_aux'
                                                    ,'usuarios.consumoUsuariosCarga' : 'usuarios.tablero11consumoUsuariosCarga'
                                                    ,'usuarios.consumoUsuariosDescarga':'usuarios.tablero11consumoUsuariosDescarga'
                                                    ,'usuarios.consumoUsuarios':'usuarios.tablero11consumoUsuarios'
                                                   })
    
except:
    pass

In [34]:
try:
    use_these_keys = ['usuarios.tablero11.siteID'
                      ,'usuarios.tablero11.nomCentroDigital'
                      , 'usuarios.tablero11.localidad'
                      , 'usuarios.tablero11.nombreDepartamento'
                      , 'usuarios.tablero11.nombreMunicipio'
                      , 'usuarios.tablero11.idBeneficiario'
                      , 'usuarios.tablero11.location'
                      , 'usuarios.tablero11.sesiones_Usuarios'
                      , 'usuarios.tablero11.usuariosConectados'
                      , 'usuarios.tablero11.consumoUsuarios'
                      , 'usuarios.tablero11.consumoUsuariosDescarga_aux'
                      , 'usuarios.tablero11.consumoUsuariosCarga_aux'
                      , 'usuarios.tablero11.consumoUsuariosDescarga'
                      , 'usuarios.tablero11.consumoUsuariosCarga'
                      , 'usuarios.tablero11.apGroup'
                      , 'usuarios.tablero11.fechaControl'
                      , 'usuarios.tablero11.fecha'
                      , 'usuarios.tablero11.anyo'
                      , 'usuarios.tablero11.mes'
                      , 'usuarios.tablero11.macRed'
                      , '@timestamp']


    usuariosConectados['@timestamp'] = now.isoformat()
    
    #Seleccionar campos del Tablero..
    
    #Cambio de variables en usuariosConectados...
    
    usuariosConectados = cambioVariable(usuariosConectados)
    
    
    def doc_generator_consumo(df,use_these_keys):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'usuariosConectados-'+ str(document['usuarios.tablero11.siteID']) + str(document['usuarios.tablero11.macRed']) + '-' + str(document['usuarios.tablero11.fechaControl']) +'-'+ str(random.randrange(10000))}",
                    "_source": filterKeys(document,use_these_keys),
                }

    usuariosConectados.fillna({'usuarios.tablero11.sesiones_Usuarios':0
                              , 'usuarios.tablero11.usuariosConectados':0
                              , 'usuarios.tablero11.consumoUsuarios':0
                              , 'usuarios.tablero11.consumoUsuariosDescarga_aux':0
                              , 'usuarios.tablero11.consumoUsuariosCarga_aux':0
                              , 'usuarios.tablero11.consumoUsuariosDescarga':0
                              , 'usuarios.tablero11.consumoUsuariosCarga':0},inplace=True)
    
    usuariosConectados.fillna("",inplace=True)
    
    salida = helpers.bulk(es, doc_generator_consumo(usuariosConectados,use_these_keys))
    
    print("Fecha: ", now,"- Usuarios conectados insertados en indice principal:",salida[0])
    
except Exception as e:  
    
    print("Fecha: ", now,"- Ningun usuario conectado para insertar en indice principal:")

Fecha:  2022-01-05 17:23:55.406389 - Usuarios conectados insertados en indice principal: 34901


### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [35]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero11',
        body = { 'jerarquia-tablero11': 'jerarquia-tablero11','tablero11.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-06-07 00:00:00
